In [1]:
using DataFrames
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using MatrixMarket

In [18]:
# File Opening
# Ordering Lines ✓
# Parsing the Lines
# Extraction of Matricies/Vectors
# Saving Individual Files

# Future Implementation
    # Folder Access and search for automation
    # GUI

function numvec(Number)
    num_vec = zeros(Number)
    for i in 1:Number
        num_vec[i] = i
    end
    num_vec
end


function find_lines_with_character(file_path::AbstractString, character::AbstractString)
    matching_lines = Vector{Int}()

    # Open the file
    file = open(file_path, "r")

    # Read the file line by line and search for the character
    line_number = 0
    for line in eachline(file)
        line_number += 1

        if contains(line, character)
            push!(matching_lines, line_number)
        end
    end

    close(file)

    return matching_lines
end



# Example usage:
file_name = "QPLIB_8938"
file_path = "QPLIB_8938\\QPLIB_8938.qplib"
character_to_search = "#"

line_list = find_lines_with_character(file_path, character_to_search);


In [19]:
#Should this be thrown into a Structure for easier computational accessiblity? 


Lines = readlines(file_path)

num_var = Int(parse(Float64,split(Lines[line_list[1]])[1]))
num_const = Int(parse(Float64,split(Lines[line_list[2]])[1]))
num_quadterms = Int(parse(Float64,split(Lines[line_list[3]])[1]))

defval_lincoeff = parse(Float64,split(Lines[line_list[4]])[1])
numndef_lincoeff = parse(Float64,split(Lines[line_list[5]])[1])

obj_const = parse(Float64,split(Lines[line_list[6]])[1])

numlin_const = parse(Float64,split(Lines[line_list[7]])[1])

val_inf = Float64(parse(BigFloat,split(Lines[line_list[8]])[1]))

def_lhs_val = Float64(parse(BigFloat,split(Lines[line_list[9]])[1]))
numndef_lhs = parse(Float64,split(Lines[line_list[10]])[1])

def_rhs_val = Float64(parse(BigFloat,split(Lines[line_list[11]])[1]))
numndef_rhs = parse(Float64,split(Lines[line_list[12]])[1])

def_var_lb = parse(BigFloat,split(Lines[line_list[13]])[1])
numndef_var_lb = parse(Float64,split(Lines[line_list[14]])[1])

def_var_ub = Float64(parse(BigFloat,split(Lines[line_list[15]])[1]))
numndef_var_ub = parse(Float64,split(Lines[line_list[16]])[1])

def_var_prime = Float64(parse(Float64,split(Lines[line_list[17]])[1]))
numndef_var_prime = parse(Float64,split(Lines[line_list[18]])[1])

def_const_dual = Float64(parse(Float64,split(Lines[line_list[19]])[1]))
numndef_const_dual = parse(Float64,split(Lines[line_list[20]])[1])

def_var_bdual = Float64(parse(Float64,split(Lines[line_list[21]])[1]))
numndef_var_bdual = parse(Float64,split(Lines[line_list[22]])[1])

numndef_var_name = parse(Float64,split(Lines[line_list[23]])[1])
numndef_const_name = parse(Float64,split(Lines[line_list[24]])[1])


0.0

In [56]:
# Implentation of a if statement to qualify whether it has anything to extractor, or if it is default values.

# Hessian Extractor ✓ <- Should not require an if statement, as this is the structure of the problem.

b_length = (line_list[4]-1)-(line_list[3])
B_cord = zeros(b_length,3)
for i in 1:b_length
    B_cord[i,:] = parse.(Float64,split(Lines[line_list[3] + i])[1:3])
end
B_sparse = sparse(B_cord[:,1],B_cord[:,2],B_cord[:,3])

B = mmwrite(string(file_name, "_B", ".mtx"),B_sparse)

# Linear Coefficients in Objective

c_length = (line_list[6]-1)-(line_list[5])

if c_length == 0

else
    c_cord = zeros(c_length,2)
    c_ones = ones(c_length)
    for i in 1:c_length
        c_cord[i,:] = parse.(Float64,split(Lines[line_list[5] + i])[1:2])
    end
    c_sparse = sparse(c_cord[:,1],c_ones,c_cord[:,2])

    c = mmwrite(string(file_name, "_c", ".mtx"),c_sparse)
end

# Constraints

cons_length = (line_list[8]-1)-(line_list[7])

if cons_length == 0

else
    cons_cord = zeros(cons_length,3)
    cons_ones = ones(cons_length)
    for i in 1:cons_length
        cons_cord[i,:] = parse.(Float64,split(Lines[line_list[7] + i])[1:3])
    end
    cons_sparse = sparse(cons_cord[:,1],cons_cord[:,2],cons_cord[:,3])

    cons = mmwrite(string(file_name, "_cons", ".mtx"),cons_sparse)
end

# Lhs Values
    # Create a Vector of Constraint Size
    # If statement for moving if the new_vector[i,:] = read_vector[i]


lhs_length = (line_list[11]-1)-(line_list[10])


if lhs_length == 0
else
    lhs_vector = hcat(numvec(num_const),fill(def_lhs_val,num_const))
    lhs_cord = zeros(lhs_length,2)
    lhs_ones = ones(num_const)
    for i in 1:lhs_length
        lhs_cord[i,:] = parse.(Float64,split(Lines[line_list[10] + i])[1:2])
    end

    for i in 1:lhs_length
        lhs_vector[Int64(lhs_cord[i,1]),2] = lhs_cord[i,2]
    end

    lhs_sparse = sparse(lhs_vector[:,1],lhs_ones,lhs_vector[:,2])

    lhs = mmwrite(string(file_name, "_lhs", ".mtx"),lhs_sparse)
end

# Rhs Values

rhs_length = (line_list[13]-1)-(line_list[12])


if rhs_length == 0
else
    rhs_vector = hcat(numvec(num_const),fill(def_rhs_val,num_const))
    rhs_cord = zeros(rhs_length,2)
    rhs_ones = ones(num_const)
    for i in 1:rhs_length
        rhs_cord[i,:] = parse.(BigFloat,split(Lines[line_list[12] + i])[1:2])
    end

    for i in 1:rhs_length
        rhs_vector[Int64(rhs_cord[i,1]),2] = rhs_cord[i,2]
    end

    rhs_sparse = sparse(rhs_vector[:,1],rhs_ones,rhs_vector[:,2])

    rhs = mmwrite(string(file_name, "_rhs", ".mtx"),rhs_sparse)
end


# lb values

lb_length = (line_list[13]-1)-(line_list[12])


if lb_length == 0
    lb_vector = hcat(numvec(num_var),fill(def_lb_val,num_var))
    lb_ones = ones(num_var)
    lb_sparse = sparse(lb_vector[:,1],lb_ones,lb_vector[:,2])
    lb = mmwrite(string(file_name, "_lb", ".mtx"),lb_sparse)
else
    lb_vector = hcat(numvec(num_var),fill(def_var_lb,num_var))
    lb_cord = zeros(lb_length,2)
    lb_ones = ones(num_var)
    for i in 1:lb_length
        lb_cord[i,:] = parse.(BigFloat,split(Lines[line_list[12] + i])[1:2])
    end

    for i in 1:lb_length
        lb_vector[Int64(lb_cord[i,1]),2] = lb_cord[i,2]
    end

    lb_sparse = sparse(lb_vector[:,1],lb_ones,lb_vector[:,2])

    lb = mmwrite(string(file_name, "_lb", ".mtx"),lb_sparse)
end

# ub values




#-----------------------------------------#

# prime 
# constraint dual 
# variable bound dual

#-----------------------------------------#

# Function Wrapper

BoundsError: BoundsError: attempt to access 4001×2 Matrix{BigFloat} at index [4002, 2]

In [54]:
lhs_vector


Inf